In [2]:
import pandas as pd
import numpy as np
import sklearn.metrics.pairwise as pw
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [90]:
df = pd.read_csv(r'BancoDeDadosCarros.csv')

In [99]:
df.sample(3)

,Make,Model,Year,Engine Fuel Type,Engine HP,Engine Cylinders,Transmission Type,Driven_Wheels,Number of Doors,Market Category,Vehicle Size,Vehicle Style,highway MPG,city mpg,Popularity,MSRP
2187,Chevrolet,Camaro,2016,premium unleaded (recommended),275.0,4.0,MANUAL,rear wheel drive,2.0,Performance,Midsize,Coupe,30,21,1385,25700
8055,Dodge,RAM 150,1992,regular unleaded,230.0,8.0,MANUAL,four wheel drive,2.0,NaN,Large,Extended Cab Pickup,15,12,1851,2000
332,Nissan,370Z,2016,premium unleaded (required),350.0,6.0,MANUAL,rear wheel drive,2.0,"Factory Tuner,High-Performance",Compact,Coupe,26,18,2009,45490


In [102]:
def get_customer_preferences(df):
    print("Olá! Vamos ajudar você a escolher um carro.")

    vehicle_styles = df['Vehicle Style'].dropna().unique()
    print("\nQual estilo de veículo você prefere?")
    for i, style in enumerate(vehicle_styles, 1):
        print(f"{i}. {style}")
    EstiloVeiculo = int(input("Escolha um número: ")) - 1
    preferred_style = vehicle_styles[EstiloVeiculo]

    AnoVeiculo = df['Year'].dropna().unique()
    print("\nQual ano de modelo você prefere?")
    for i, year in enumerate(sorted(AnoVeiculo), 1):
        print(f"{i}. {year}")
    AnoEscolhido = int(input("Escolha um número: ")) - 1
    preferred_year = sorted(AnoVeiculo)[AnoEscolhido]

    TipoCombustivel = df['Engine Fuel Type'].dropna().unique()
    print("\nQual tipo de combustível você prefere?")
    for i, fuel in enumerate(TipoCombustivel, 1):
        print(f"{i}. {fuel}")
    CombEscolhido = int(input("Escolha um número: ")) - 1
    preferred_fuel = TipoCombustivel[CombEscolhido]

    cylinders = df['Engine Cylinders'].dropna().unique()
    cylinders = cylinders[~pd.isna(cylinders)]
    print("\nQuantos cilindros você prefere?")
    for i, cylinder in enumerate(sorted(cylinders), 1):
        print(f"{i}. {cylinder}")
    cylinder_choice = int(input("Escolha um número: ")) - 1
    preferred_cylinders = sorted(cylinders)[cylinder_choice]

    return preferred_style, preferred_year, preferred_fuel, preferred_cylinders

preferences = get_customer_preferences(df)
user_preferences = {
    'Vehicle Style': preferences[0],
    'Year': preferences[1],
    'Engine Fuel Type': preferences[2],
    'Engine Cylinders': preferences[3]
}

user_df = pd.DataFrame([user_preferences])
df_combined = pd.concat([df, user_df], ignore_index=True)
df_combined = df_combined.dropna(subset=['Vehicle Style', 'Engine Fuel Type', 'Year', 'Engine Cylinders'])
label_encoders = {}
for column in ['Vehicle Style', 'Engine Fuel Type']:
    le = LabelEncoder()
    df_combined[column] = le.fit_transform(df_combined[column])

numeric_cols = ['Year', 'Engine Cylinders']
df_numeric = df_combined[numeric_cols].copy()
df_numeric = df_numeric.dropna()
if df_numeric.isnull().values.any():
    print("Atenção: Existem valores NaN após a limpeza.")
cosine_sim = cosine_similarity(df_numeric)
car_indices = range(len(df_combined) - 1)
user_vector = cosine_sim[-1][car_indices]
similarity_df = pd.DataFrame({
    'Car Index': car_indices,
    'Similarity': user_vector
})

top_cars = similarity_df.nlargest(5, 'Similarity')
print("\nOs 5 melhores carros que atendem às suas preferências:")
for index in top_cars['Car Index']:
    car_info = df.iloc[index]
    print(f"Carro: {car_info['Make']} {car_info['Model']}, Ano: {car_info['Year']}, "
          f"Tipo: {car_info['Engine Fuel Type']}, Cilindros: {car_info['Engine Cylinders']}, "
          f"Transmissão: {car_info['Transmission Type']}, Portas: {car_info['Number of Doors']}, "
          f"Economia Cidade: {car_info['city mpg']}Km, Economia Estrada: {car_info['highway MPG']}Km, "
          f"Preço: R${car_info['MSRP']},00")


Olá! Vamos ajudar você a escolher um carro.

Qual estilo de veículo você prefere?
1. Coupe
2. Convertible
3. Sedan
4. Wagon
5. 4dr Hatchback
6. 2dr Hatchback
7. 4dr SUV
8. Passenger Minivan
9. Cargo Minivan
10. Crew Cab Pickup
11. Regular Cab Pickup
12. Extended Cab Pickup
13. 2dr SUV
14. Cargo Van
15. Convertible SUV
16. Passenger Van
Escolha um número: 7

Qual ano de modelo você prefere?
1. 1990
2. 1991
3. 1992
4. 1993
5. 1994
6. 1995
7. 1996
8. 1997
9. 1998
10. 1999
11. 2000
12. 2001
13. 2002
14. 2003
15. 2004
16. 2005
17. 2006
18. 2007
19. 2008
20. 2009
21. 2010
22. 2011
23. 2012
24. 2013
25. 2014
26. 2015
27. 2016
28. 2017
Escolha um número: 28

Qual tipo de combustível você prefere?
1. premium unleaded (required)
2. regular unleaded
3. premium unleaded (recommended)
4. flex-fuel (unleaded/E85)
5. diesel
6. electric
7. flex-fuel (premium unleaded recommended/E85)
8. natural gas
9. flex-fuel (premium unleaded required/E85)
10. flex-fuel (unleaded/natural gas)
Escolha um número: 4



In [106]:
df = df.dropna(subset=['Vehicle Style', 'Engine Fuel Type', 'Engine Cylinders', 'Year'])
def get_customer_preferences(df):
    print("Olá! Vamos ajudar você a escolher um carro.")
    vehicle_styles = df['Vehicle Style'].unique()
    print("\nQual estilo de veículo você prefere?")
    for i, style in enumerate(vehicle_styles, 1):
        print(f"{i}. {style}")
    style_choice = int(input("Escolha um número: ")) - 1
    preferred_style = vehicle_styles[style_choice]
    filtered_years = df[df['Vehicle Style'] == preferred_style]['Year'].unique()
    print("\nQual ano de modelo você prefere?")
    for i, year in enumerate(sorted(filtered_years), 1):
        print(f"{i}. {year}")
    year_choice = int(input("Escolha um número: ")) - 1
    preferred_year = sorted(filtered_years)[year_choice]
    filtered_fuel_types = df[(df['Vehicle Style'] == preferred_style) & (df['Year'] == preferred_year)]['Engine Fuel Type'].unique()
    print("\nQual tipo de combustível você prefere?")
    for i, fuel in enumerate(filtered_fuel_types, 1):
        print(f"{i}. {fuel}")
    fuel_choice = int(input("Escolha um número: ")) - 1
    preferred_fuel = filtered_fuel_types[fuel_choice]
    filtered_cylinders = df[(df['Vehicle Style'] == preferred_style) & (df['Year'] == preferred_year) & (df['Engine Fuel Type'] == preferred_fuel)]['Engine Cylinders'].unique()
    print("\nQuantos cilindros você prefere?")
    for i, cylinder in enumerate(sorted(filtered_cylinders), 1):
        print(f"{i}. {cylinder}")
    cylinder_choice = int(input("Escolha um número: ")) - 1
    preferred_cylinders = sorted(filtered_cylinders)[cylinder_choice]
    transmissions = df['Transmission Type'].unique()
    print("\nQual tipo de transmissão você prefere?")
    for i, transmission in enumerate(transmissions, 1):
        print(f"{i}. {transmission}")
    transmission_choice = int(input("Escolha um número: ")) - 1
    preferred_transmission = transmissions[transmission_choice]
    return preferred_style, preferred_year, preferred_fuel, preferred_cylinders, preferred_transmission
preferences = get_customer_preferences(df)
preferred_style, preferred_year, preferred_fuel, preferred_cylinders, preferred_transmission = preferences
user_preferences = pd.DataFrame({
    'Vehicle Style': [preferred_style],
    'Year': [preferred_year],
    'Engine Fuel Type': [preferred_fuel],
    'Engine Cylinders': [preferred_cylinders],
    'Transmission Type': [preferred_transmission]
})
df_combined = pd.concat([df, user_preferences], ignore_index=True)
label_encoders = {}
for column in ['Vehicle Style', 'Engine Fuel Type', 'Transmission Type']:
    le = LabelEncoder()
    df_combined[column] = le.fit_transform(df_combined[column].fillna('unknown'))
    label_encoders[column] = le
df_combined['Engine Cylinders'] = df_combined['Engine Cylinders'].fillna(df_combined['Engine Cylinders'].median())
df_combined['Year'] = df_combined['Year'].fillna(df_combined['Year'].median())
df_numeric = df_combined[['Engine Cylinders', 'Year']].astype(float)
cosine_sim = cosine_similarity(df_numeric)
car_indices = np.argsort(cosine_sim[-1])[:-6:-1]
print("\nOs 5 melhores carros que atendem às suas preferências:")
if car_indices.size == 0:
    print("Desculpe, não encontramos carros que atendam às suas preferências.")
else:
    for index in car_indices:
        if index < len(df):
            car_info = df.iloc[index]
            print(f"Carro: {car_info['Make']} {car_info['Model']}, Ano: {car_info['Year']}, Tipo: {car_info['Engine Fuel Type']}, Cilindros: {car_info['Engine Cylinders']}, Transmissão: {car_info['Transmission Type']}, Portas: {car_info['Number of Doors']}, Economia Cidade: {car_info['city mpg']}Km, Economia Estrada: {car_info['highway MPG']}Km, Preço: R${car_info['MSRP']:,.2f}")


Olá! Vamos ajudar você a escolher um carro.

Qual estilo de veículo você prefere?
1. Coupe
2. Convertible
3. Sedan
4. Wagon
5. 4dr Hatchback
6. 2dr Hatchback
7. 4dr SUV
8. Passenger Minivan
9. Cargo Minivan
10. Crew Cab Pickup
11. Regular Cab Pickup
12. Extended Cab Pickup
13. 2dr SUV
14. Cargo Van
15. Convertible SUV
16. Passenger Van
Escolha um número: 2

Qual ano de modelo você prefere?
1. 1990
2. 1991
3. 1992
4. 1993
5. 1994
6. 1995
7. 1996
8. 1997
9. 1998
10. 1999
11. 2000
12. 2001
13. 2002
14. 2003
15. 2004
16. 2005
17. 2006
18. 2007
19. 2008
20. 2009
21. 2010
22. 2011
23. 2012
24. 2013
25. 2014
26. 2015
27. 2016
28. 2017
Escolha um número: 28

Qual tipo de combustível você prefere?
1. premium unleaded (recommended)
2. premium unleaded (required)
3. flex-fuel (premium unleaded recommended/E85)
Escolha um número: 3

Quantos cilindros você prefere?
1. 4.0
Escolha um número: 1

Qual tipo de transmissão você prefere?
1. MANUAL
2. AUTOMATIC
3. AUTOMATED_MANUAL
4. DIRECT_DRIVE
5. UNKNO